In [133]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
!conda install -c conda-forge folium=0.5.0 --yes
import folium
# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



In [157]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
## PART 1 SCRAPPING NEIGHBORHOOD INFO 

#define headers for the request
headers = requests.utils.default_headers()
headers.update({"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36"})

In [4]:
#Wikipedia link for the information to scrap
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')


In [5]:

#Finding columns name html style
columns_name = str(soup.table.find_all("th")).replace("\n","")

#cleaning tags
columns_string=re.findall('<th>(.+?)</th>',columns_name)

#Transform in a Series to create data frame
columns_name = pd.Series(list(columns_string))


In [6]:

#finding dataframe values
values = str(soup.table.find_all("td")).replace("\n","")
values = re.findall('<td>(.+?)</td>',values)

#finding Neighbourhood clean name
neigh = []
for value in values:
    if len(re.findall('>(.+?)</a>',str(value)))<1:
        neigh.append(value)
    else:
        a = re.findall('>(.+?)<',str(value)) #result type is a list
        neigh.append(a[0])

#We have a list with all values, now we are going to create 3 Series for the dataframe values (1 for Postcodes, 1 for Borough and 1 for Neighbourhoods)
#3 empty list
pc = [] #Postcode
br = [] #Borough
ne = [] #Neighbourhoods

for i in range(len(neigh)):
    if i == 0:
        pc.append(neigh[i])
        br.append(neigh[i+1])
        ne.append(neigh[i+2])
    elif i in range(int(len(neigh)/3)):
        pc.append(neigh[0+i*3])
        br.append(neigh[1+i*3])
        ne.append(neigh[2+i*3])
        

In [7]:
#Creating DataFrame with the data scrapped
d = {"PostalCode":pc,
     columns_name[1]:br,
     "Neighborhood":ne}
df = pd.DataFrame(d)

#Selecting Index with Borough == Not assigned
dropIndex = df[df['Borough']=="Not assigned"].index
#dropping Not Assigned Borough values

df.drop(dropIndex,inplace=True)

In [8]:
#Grouping by Postal Code and Borough
df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
#printing number of rows
df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df.shape

(103, 3)

In [10]:
## PART 2 ADD LATITUDE AND LONGITUDE

#Lets Start with the second part, Getting lat and lon from each Postal Code
#URl to csv file to get Lat and Long

csv = 'https://cocl.us/Geospatial_data'
location = pd.read_csv(csv)


In [11]:
neighborhoods = pd.concat([df,location], axis=1, join='inner')

In [12]:
neighborhoods.drop(columns=['Postal Code'],inplace=True)
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [13]:
#PART 3 EXPLORE AND CLUSTERING

#There is a diference between the job done with New York data and the Toronto Data. Toronto has many Neighborhoods in the same borough, 
#and the principal geolocation is determined by the PostalCode. I going to incluide PostCode and Borough in each analysis.

#Creating Toronto Map

latitude = 43.6532
longitude = -79.3832
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postcode in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood'], neighborhoods['PostalCode']):
    label = '{} {}: {}'.format(postcode, borough, neighborhood )
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [14]:
#Creating a dataframe only with borough that contains the word "TORONTO"
Neigh = neighborhoods[neighborhoods['Borough'].str.contains('Toronto',case=False)].reset_index()

Neigh.drop(columns='index',inplace=True)

In [15]:
Neigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
#Creating a Map with only the borough that contains de word "TORONTO"
latitude = 43.6532
longitude = -79.3832
map_Toronto2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood, postcode in zip(Neigh['Latitude'], Neigh['Longitude'], Neigh['Borough'], Neigh['Neighborhood'], Neigh['PostalCode']):
    label = '{} {}: {}'.format(postcode, borough, neighborhood )
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto2)  
    
map_Toronto2

In [17]:
#foursquare credentials
CLIENT_ID = 'JRRZNIVCUYW3DIXKI34LPFI11ZQID4MOXAXI4VVIFH3GXBFM' #Foursquare ID
CLIENT_SECRET = 'GHZGQEA2VDYJYEXAJAPQI4SWSW241WXS5QEQDSSS4SERFPL3' #Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



In [18]:
#Funtion to scrap the foursquare information, I going to include PostalCode and Borough. Because many neighborhood has the same location of one PostalCode.

def getNearbyVenues(postcode, borough, neigh, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postcode, borough, neigh, lat, lng in zip(postcode, borough, neigh, latitudes, longitudes):
        print('{} {}: {}'.format(postcode,borough,neigh))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode,
            borough,
            neigh,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                  'Borough',
                  'Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
Toronto_venues = getNearbyVenues(postcode=Neigh['PostalCode'], 
                                 borough=Neigh['Borough'], 
                                 neigh=Neigh['Neighborhood'], 
                                 latitudes=Neigh['Latitude'], 
                                 longitudes=Neigh['Longitude'],
                                 radius=500)

M4E East Toronto: The Beaches
M4K East Toronto: The Danforth West,Riverdale
M4L East Toronto: The Beaches West,India Bazaar
M4M East Toronto: Studio District
M4N Central Toronto: Lawrence Park
M4P Central Toronto: Davisville North
M4R Central Toronto: North Toronto West
M4S Central Toronto: Davisville
M4T Central Toronto: Moore Park,Summerhill East
M4V Central Toronto: Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
M4W Downtown Toronto: Rosedale
M4X Downtown Toronto: Cabbagetown,St. James Town
M4Y Downtown Toronto: Church and Wellesley
M5A Downtown Toronto: Harbourfront
M5B Downtown Toronto: Ryerson,Garden District
M5C Downtown Toronto: St. James Town
M5E Downtown Toronto: Berczy Park
M5G Downtown Toronto: Central Bay Street
M5H Downtown Toronto: Adelaide,King,Richmond
M5J Downtown Toronto: Harbourfront East,Toronto Islands,Union Station
M5K Downtown Toronto: Design Exchange,Toronto Dominion Centre
M5L Downtown Toronto: Commerce Court,Victoria Hotel
M5N Central Toronto: 

In [21]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1721, 9)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [22]:
Toronto_venues.groupby(['PostalCode','Borough','Neighborhood']).count()

,,,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West,Riverdale",41,41,41,41,41,41
M4L,East Toronto,"The Beaches West,India Bazaar",18,18,18,18,18,18
M4M,East Toronto,Studio District,42,42,42,42,42,42
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,20,20,20,20,20,20
M4S,Central Toronto,Davisville,37,37,37,37,37,37
M4T,Central Toronto,"Moore Park,Summerhill East",2,2,2,2,2,2


In [62]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [110]:
#Now we create a dataframe with the venues as columns
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# adding PostalCode, Borough and Neighborhood column back to dataframe
#We change the name of the columns neighborhood to neighbohoods because there is one venue with the same name
Toronto_onehot[['PostalCode','Borough','Neighborhoods']] = Toronto_venues[['PostalCode','Borough','Neighborhood']] 

# move PostalCode, Borough and Neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-3]]+ [Toronto_onehot.columns[-2]] +[Toronto_onehot.columns[-1]]+ list(Toronto_onehot.columns[:-3])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West,Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
#Lets group by PostalCode, Borough and Neighborhoods
Toronto_grouped = Toronto_onehot.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()
Toronto_grouped

,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.02439,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.024390
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.00,0.023810
4,M4N,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.050000
7,M4S,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,Central Toronto,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.50000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.00,0.000000


In [115]:
#Lets create a funtion to get the most common venues for each group.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [131]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode','Borough','Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[['PostalCode','Borough','Neighborhoods']] = Toronto_grouped[['PostalCode','Borough','Neighborhoods']]

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 3:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",Park,Pizza Place,Sushi Restaurant,Brewery,Fish & Chips Shop,Pub,Fast Food Restaurant,Italian Restaurant,Steakhouse,Movie Theater
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,Park,Hotel,Sandwich Place,Food & Drink Shop,Department Store,Breakfast Spot,Gym,Cosmetics Shop,Ethiopian Restaurant,Electronics Store
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Mexican Restaurant,Dessert Shop,Café,Diner,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant
7,M4S,Central Toronto,Davisville,Pizza Place,Gym,Dessert Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Brewery,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park,Summerhill East",Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Coffee Shop,Pub,Pizza Place,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Supermarket,Sushi Restaurant,Fried Chicken Joint


In [173]:
#Lets Create Cluster Using information 
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop(['Neighborhoods','PostalCode','Borough'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 0, 0, 4, 0], dtype=int32)

In [154]:
#Lets include the Clusters with all the information
Toronto_merged = Neigh

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index(['PostalCode','Borough','Neighborhoods']), on=['PostalCode','Borough','Neighborhood'])

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Pizza Place,Sushi Restaurant,Brewery,Fish & Chips Shop,Pub,Fast Food Restaurant,Italian Restaurant,Steakhouse,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [174]:
# create a map with the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [153]:
Neigh

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,4
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0


In [171]:
Toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Pizza Place,Sushi Restaurant,Brewery,Fish & Chips Shop,Pub,Fast Food Restaurant,Italian Restaurant,Steakhouse,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Park,Hotel,Sandwich Place,Food & Drink Shop,Department Store,Breakfast Spot,Gym,Cosmetics Shop,Ethiopian Restaurant,Electronics Store
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Mexican Restaurant,Dessert Shop,Café,Diner,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Gym,Dessert Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Brewery,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,4,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0,Coffee Shop,Pub,Pizza Place,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Supermarket,Sushi Restaurant,Fried Chicken Joint
